# Agente utilizando LangChain y Antropic

Este notebook asume que estás trabajando con Python 3.12.4 en [VsCode](https://code.visualstudio.com/).

<br />

Requisitos:  
- Para la ejecución de este nbotebook se requere acceso a las siguientes APIs:
    - [LangChain](https://smith.langchain.com/settings)
    - [Antropic](https://console.anthropic.com/)
    - [Tavily](https://tavily.com/)
- Reemplaza los valores de `ANTHROPIC_API_KEY` y `TAVILY_API_KEY` en el archivo `.env` de este proyecto.  

    > NOTA: Esta es informacón sensible, asegurate de no versionarla en el repositorio.
- Instala las dependencias usando `pip install -r requirements.txt`

<br />

## Carga variables de ambiente desde el archivo .env

In [1]:
from dotenv import load_dotenv
import os


load_dotenv() 

assert os.environ['LANGCHAIN_API_KEY'] != 'API_KEY', 'Remplaza el valor de la variable LANGCHAIN_API_KEY en el archivo .env'
assert os.environ['ANTHROPIC_API_KEY'] != 'API_KEY', 'Remplaza el valor de la variable ANTHROPIC_API_KEY en el archivo .env'
assert os.environ['TAVILY_API_KEY'] != 'API_KEY', 'Remplaza el valor de la variable TAVILY_API_KEY en el archivo .env'

## Definir herramientas

Las herramientas son las que permiten a los agentes interactuar con otros servicios. Para este este ejemplo utilzaremos [Tavily](https://tavily.com/), un motor de busqueda optimizado para modelos de lenguaje.

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults


buscador = TavilySearchResults(max_results=2)

resultados_busqueda = buscador.invoke("¿Cómo está el clima en Monterrey, México?",)
print(resultados_busqueda)

[{'url': 'https://www.meteored.mx/clima_Monterrey-America+Norte-Mexico-Nuevo+Leon-MMMY-1-21045.html', 'content': 'Clima en Monterrey con el estado del tiempo a 14 días. Los datos sobre el Tiempo, temperatura, velocidad del viento, la humedad, la cota de nieve, presión, etc . ... Se esperan lluvias y tormentas eléctricas en varios estados de México. Hace 7 horas El tiempo en Mexicali, Tijuana y Baja California, 14 de septiembre: rachas de hasta 50 km/h ...'}, {'url': 'https://weather.com/es-MX/tiempo/10dias/l/Monterrey+Nuevo+León?canonicalCityId=c641151b28f60389898f58aba529d9ec306c2ccb04ad699216c44ee7c172cc75', 'content': 'Prepárate con el pronóstico para los próximos 10 días más preciso para Monterrey, Nuevo León. Consulta la temperatura máxima y mínima y la probabilidad de lluvia en The Weather Channel y ...'}]


## Carga del modelo

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage


modelo = ChatAnthropic(model="claude-3-5-sonnet-20240620")

response = modelo.invoke([HumanMessage(content="¡Hola!")])
response.content

'¡Hola! ¿Cómo estás? ¿En qué puedo ayudarte hoy?'

## Agregar herramientas al modelo

In [4]:
herramientas = [
    buscador,
    # otras herramientas
]

modelo_con_herramientas = modelo.bind_tools(herramientas)

In [5]:
# Esta llamada NO requiere herramientas.
response = modelo_con_herramientas.invoke([HumanMessage(content="¡Hola!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: ¡Hola! Bienvenido. ¿Cómo puedo ayudarte hoy? Estoy aquí para responder tus preguntas o asistirte en lo que necesites. ¿Hay algún tema en particular sobre el que quieras hablar o alguna información que estés buscando?
ToolCalls: []


In [6]:
# Esta llamada SI requiere herramientas.
response = modelo_con_herramientas.invoke([HumanMessage(content="¿Cómo está el clima en Monterrey, México?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: [{'text': 'Para obtener información actualizada sobre el clima en Monterrey, México, necesitaré usar la herramienta de búsqueda disponible. Permítame hacer una consulta para obtener esa información para usted.', 'type': 'text'}, {'id': 'toolu_01LXXtpEtS8myRd3obohd8yX', 'input': {'query': 'clima actual en Monterrey, México'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'clima actual en Monterrey, México'}, 'id': 'toolu_01LXXtpEtS8myRd3obohd8yX', 'type': 'tool_call'}]


# Creación del agente

In [7]:
from langgraph.prebuilt import create_react_agent


agente = create_react_agent(modelo, herramientas)

In [8]:
# Esta llamada NO requiere herramientas.
response = agente.invoke({"messages": [HumanMessage(content="¡Hola!")]})
response["messages"]

[HumanMessage(content='¡Hola!', additional_kwargs={}, response_metadata={}, id='9d5cfac6-1547-4767-a24d-737fa909cf3f'),
 AIMessage(content='¡Hola! Bienvenido. ¿Cómo puedo ayudarte hoy? Estoy aquí para asistirte con cualquier pregunta o información que necesites. ¿Hay algo en particular sobre lo que te gustaría saber o conversar?', additional_kwargs={}, response_metadata={'id': 'msg_01UmZwDvXe9rxBGQ1aQjzzWK', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 404, 'output_tokens': 73}}, id='run-de5232af-0a73-48a1-b3a4-36f79ba0e31b-0', usage_metadata={'input_tokens': 404, 'output_tokens': 73, 'total_tokens': 477})]

In [9]:
# Esta llamada SI requiere herramientas.
response = agente.invoke({"messages": [HumanMessage(content="¿Cómo está el clima en Monterrey, México?")]})
response["messages"]

[HumanMessage(content='¿Cómo está el clima en Monterrey, México?', additional_kwargs={}, response_metadata={}, id='4e46a044-a4ee-47c1-8ac4-5eff175b023a'),
 AIMessage(content=[{'text': 'Para obtener información actualizada sobre el clima en Monterrey, México, necesitaré usar la herramienta de búsqueda disponible. Permítame buscar esa información para usted.', 'type': 'text'}, {'id': 'toolu_01CLeYTxZWndjt4BtxHrum6Q', 'input': {'query': 'clima actual en Monterrey, México'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01Hnc6fENxAnqPHNNNKtBqeH', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 113}}, id='run-b619caa8-0dbe-4cee-9d70-c8b5fe466099-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'clima actual en Monterrey, México'}, 'id': 'toolu_01CLeYTxZWndjt4BtxHrum6Q', 'type': 'tool_call'}], usage_metadata={'input

In [10]:
# Proceso paso a paso
for chunk in agente.stream(
    {"messages": [HumanMessage(content="¿Cómo está el clima en Monterrey, México?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': 'Para responder a su pregunta sobre el clima en Monterrey, México, necesitaré buscar información actualizada. Permítame utilizar una herramienta de búsqueda para obtener los datos más recientes.', 'type': 'text'}, {'id': 'toolu_017KVEYbgFt8ySsNGj9eViwJ', 'input': {'query': 'clima actual en Monterrey, México'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01DE61o7gL3D2DJ4wPUQ689N', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 117}}, id='run-afdbe624-fd5e-470a-bdb1-ebcdde63ba07-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'clima actual en Monterrey, México'}, 'id': 'toolu_017KVEYbgFt8ySsNGj9eViwJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 414, 'output_tokens': 117, 'total_tokens': 531})]}}
----
{'tools': {'messages': [ToolMessage(content='